# Capstone 2 - Lending Club Loan Defaults Prediction¶

# Modeling


Modeling is the fifth step in the Data Science Method.  The following will be performed in this step:

1. Fit Models with Training Data Set
2. Review Model Outcomes — Iterate over additional models as needed.
3. Identify the Final Model

In [1]:
#load python packages
import os
import pandas as pd
import pandas.api.types as ptypes
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings # For handling error messages.
#warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
# Read data output created in the EDA step
df = pd.read_csv("../EDA/data/Cap2_step3_output.csv", dtype={'verification_status_joint':object,'hardship_type':object, 'hardship_reason':object, 'hardship_status':object, 'hardship_loan_status':object, 'settlement_status':object}, index_col="Unnamed: 0")
df.head(5)

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,initial_list_status,application_type,verification_status_joint,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_loan_status,debt_settlement_flag,settlement_status
0,2500,13.56,55000.0,18.24,0.0,1.0,0.0,45.0,1.0,4341,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
1,30000,18.94,90000.0,26.52,0.0,0.0,71.0,75.0,1.0,12315,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
2,5000,17.97,59280.0,10.51,0.0,0.0,0.0,0.0,0.0,4599,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
3,4000,18.94,92000.0,16.74,0.0,0.0,0.0,0.0,0.0,5468,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
4,30000,16.14,57250.0,26.35,0.0,0.0,0.0,0.0,0.0,829,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN


In [3]:
df.shape

(2150627, 87)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2150627 entries, 0 to 2260667
Data columns (total 87 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   loan_amnt                                   int64  
 1   int_rate                                    float64
 2   annual_inc                                  float64
 3   dti                                         float64
 4   delinq_2yrs                                 float64
 5   inq_last_6mths                              float64
 6   mths_since_last_delinq                      float64
 7   mths_since_last_record                      float64
 8   pub_rec                                     float64
 9   revol_bal                                   int64  
 10  revol_util                                  float64
 11  out_prncp                                   float64
 12  total_pymnt                                 float64
 13  total_rec_int              

In [5]:
df.describe()

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_dpd,orig_projected_additional_accrued_interest,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,...,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06
mean,1.455534e+04,1.315755e+01,6.911525e+04,1.910219e+01,3.019622e-01,5.717709e-01,1.680994e+01,1.166151e+01,1.972904e-01,1.522785e+04,...,3.856550e-03,6.066408e-01,1.407915e-02,7.129102e-01,6.392043e-02,1.664888e+00,8.860760e-01,7.394659e+01,7.149019e-01,1.966361e-01
std,8.859252e+03,4.824323e+00,3.159676e+04,1.439575e+01,8.613410e-01,8.803719e-01,2.308542e+01,2.867152e+01,5.606472e-01,1.705110e+04,...,9.259225e-02,5.605731e+00,2.050347e-01,1.360392e+01,1.144538e+00,3.550980e+01,1.851189e+01,7.449337e+02,5.868766e+00,1.883835e+00
min,5.000000e+02,5.310000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+03,9.670000e+00,4.500000e+04,1.219000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.810000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.200000e+04,1.269000e+01,6.400000e+04,1.815000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.096600e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.000000e+04,1.599000e+01,8.800000e+04,2.476000e+01,0.000000e+00,1.000000e+00,3.000000e+01,0.000000e+00,0.000000e+00,1.927500e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.000000e+04,3.099000e+01,1.635000e+05,9.990000e+02,5.800000e+01,3.300000e+01,2.260000e+02,1.290000e+02,8.600000e+01,9.597540e+05,...,2.300000e+01,1.850000e+02,3.000000e+00,9.439400e+02,3.700000e+01,2.680890e+03,1.407860e+03,3.360100e+04,5.213500e+02,1.810000e+02


In [6]:
df['zip_code'].value_counts()

112xx    22865
945xx    21787
750xx    21749
606xx    20025
300xx    19156
         ...  
375xx        1
521xx        1
733xx        1
663xx        1
568xx        1
Name: zip_code, Length: 955, dtype: int64

## 'loan_status' is the response variable.
## Plan to predict fully paid and charged off status
## Features of interest are: 
'tot_cur_bal', 'annual_inc_joint', 'annual_inc', 'mths_since_last_record', 'mths_since_last_major_derog', 'collections_12_mths_ex_med', 'pub_rec_bankruptcies', 'tax_liens', '
## Features to not consider are
'title','zip_code','addr_state'



In [7]:
df_sel = df.drop(['title', 'zip_code', 'addr_state'], axis=1)

''' 
# Check these values in df_sel

loan_amnt                                   
annual_inc   
revol_bal  
tot_cur_bal
total_bal_il
max_bal_bc                                  
avg_cur_bal  
tax_liens      
total_bal_ex_mort   
total_bc_limit    
total_il_high_credit_limit     
revol_bal_joint        
hardship_amount      
hardship_last_payment_amount      
settlement_amount                  
'''

In [8]:
df_sel['loan_amnt'].value_counts()

10000    183149
20000    123816
12000    118794
15000    118790
5000      83452
          ...  
725           1
550           1
950           1
850           1
750           1
Name: loan_amnt, Length: 1571, dtype: int64

In [9]:
df_sel['annual_inc'].value_counts()

60000.0     87189
50000.0     76355
65000.0     64903
70000.0     62078
80000.0     59833
            ...  
44271.0         1
44269.0         1
94657.0         1
94659.0         1
131070.0        1
Name: annual_inc, Length: 83915, dtype: int64

In [10]:
df_sel['revol_bal'].value_counts()

0         11980
8           209
2           163
10          159
5235        158
          ...  
94025         1
96074         1
85837         1
110417        1
109518        1
Name: revol_bal, Length: 88143, dtype: int64

In [11]:
df_sel['tot_cur_bal'].value_counts()

0.0         69031
14186.0        41
20275.0        39
22831.0        38
23442.0        38
            ...  
309690.0        1
309689.0        1
309686.0        1
509276.0        1
381300.0        1
Name: tot_cur_bal, Length: 447363, dtype: int64

In [12]:
df_sel['total_bal_il'].value_counts()

0.0         986116
10000.0         98
15000.0         90
5000.0          83
5500.0          76
             ...  
339145.0         1
339113.0         1
339092.0         1
339074.0         1
84262.0          1
Name: total_bal_il, Length: 152044, dtype: int64

In [13]:
df_sel['max_bal_bc'].value_counts()

0.0         866592
8.0            544
3000.0         522
2000.0         495
4000.0         437
             ...  
27465.0          1
27460.0          1
27456.0          1
27453.0          1
131035.0         1
Name: max_bal_bc, Length: 30086, dtype: int64

In [14]:
df_sel['avg_cur_bal'].value_counts()

0.0         69048
2277.0        269
2442.0        267
2148.0        265
2025.0        265
            ...  
72094.0         1
72092.0         1
72087.0         1
72086.0         1
131042.0        1
Name: avg_cur_bal, Length: 80280, dtype: int64

In [15]:
df_sel['tax_liens'].value_counts()

0.0     2092608
1.0       39790
2.0       10776
3.0        3866
4.0        1673
5.0         851
6.0         443
7.0         204
8.0         135
9.0          81
10.0         62
11.0         32
12.0         19
13.0         12
14.0         11
15.0         11
17.0          7
18.0          6
22.0          5
21.0          5
16.0          5
19.0          2
20.0          2
61.0          2
24.0          2
46.0          2
45.0          2
37.0          1
53.0          1
52.0          1
48.0          1
39.0          1
34.0          1
85.0          1
29.0          1
28.0          1
27.0          1
25.0          1
23.0          1
63.0          1
Name: tax_liens, dtype: int64

In [16]:
df_sel['total_bal_ex_mort'].value_counts()

0.0         49942
24214.0        57
23068.0        57
20275.0        57
19095.0        56
            ...  
320651.0        1
194012.0        1
194009.0        1
320659.0        1
125310.0        1
Name: total_bal_ex_mort, Length: 194363, dtype: int64

In [17]:
df_sel['total_bc_limit'].value_counts()

0.0        73131
5000.0     16656
6000.0     14857
10000.0    14392
7000.0     14119
           ...  
38144.0        1
38143.0        1
38132.0        1
38130.0        1
29778.0        1
Name: total_bc_limit, Length: 18408, dtype: int64

In [18]:
df_sel['total_il_high_credit_limit'].value_counts()

0.0         324857
10000.0      13083
15000.0       9912
5000.0        8071
20000.0       7998
             ...  
211627.0         1
211619.0         1
211611.0         1
211608.0         1
194698.0         1
Name: total_il_high_credit_limit, Length: 179974, dtype: int64

In [19]:
df_sel['revol_bal_joint'].value_counts()

0.0         2045053
20804.0          10
21072.0           9
11771.0           9
10041.0           9
             ...   
29454.0           1
29455.0           1
29456.0           1
29458.0           1
130981.0          1
Name: revol_bal_joint, Length: 55621, dtype: int64

In [20]:
df_sel['hardship_amount'].value_counts()

0.00      2140534
94.59           5
48.56           5
53.05           5
69.90           5
           ...   
101.21          1
482.40          1
156.07          1
568.60          1
420.62          1
Name: hardship_amount, Length: 8553, dtype: int64

In [21]:
df_sel['hardship_last_payment_amount'].value_counts()

0.00      2140534
0.02           25
0.03           25
0.12           23
0.11           22
           ...   
482.10          1
575.35          1
157.82          1
73.23           1
373.61          1
Name: hardship_last_payment_amount, Length: 8388, dtype: int64

In [22]:
df_sel['settlement_amount'].value_counts()

0.00        2118442
5000.00          61
4000.00          49
3000.00          48
6000.00          47
             ...   
10857.25          1
10858.00          1
1048.80           1
1600.01           1
6226.00           1
Name: settlement_amount, Length: 21005, dtype: int64

In [23]:
# Create a list of object columns.  Exclude response variable 'loan_status'
cols_obj = []
for col in df_sel.columns:
    if df_sel[col].dtype == object:
        cols_obj.append(col)
cols_obj.remove('loan_status')
cols_obj

['term',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'pymnt_plan',
 'initial_list_status',
 'application_type',
 'verification_status_joint',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_loan_status',
 'debt_settlement_flag',
 'settlement_status']

In [24]:
df_sel = pd.concat([df_sel.drop(cols_obj, axis=1), pd.get_dummies(df_sel[cols_obj])], axis=1)
df_sel.head(5)

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,hardship_loan_status_Current,hardship_loan_status_In Grace Period,hardship_loan_status_Issued,hardship_loan_status_Late (16-30 days),hardship_loan_status_Late (31-120 days),debt_settlement_flag_N,debt_settlement_flag_Y,settlement_status_ACTIVE,settlement_status_BROKEN,settlement_status_COMPLETE
0,2500,13.56,55000.0,18.24,0.0,1.0,0.0,45.0,1.0,4341,...,0,0,0,0,0,1,0,0,0,0
1,30000,18.94,90000.0,26.52,0.0,0.0,71.0,75.0,1.0,12315,...,0,0,0,0,0,1,0,0,0,0
2,5000,17.97,59280.0,10.51,0.0,0.0,0.0,0.0,0.0,4599,...,0,0,0,0,0,1,0,0,0,0
3,4000,18.94,92000.0,16.74,0.0,0.0,0.0,0.0,0.0,5468,...,0,0,0,0,0,1,0,0,0,0
4,30000,16.14,57250.0,26.35,0.0,0.0,0.0,0.0,0.0,829,...,0,0,0,0,0,1,0,0,0,0


In [25]:
# Using a sample size of 200,000 for modeling, due to Memory error when using all 2 million observations.
df_sample = df_sel.sample(200000)
df_sample.head()

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,hardship_loan_status_Current,hardship_loan_status_In Grace Period,hardship_loan_status_Issued,hardship_loan_status_Late (16-30 days),hardship_loan_status_Late (31-120 days),debt_settlement_flag_N,debt_settlement_flag_Y,settlement_status_ACTIVE,settlement_status_BROKEN,settlement_status_COMPLETE
1839955,6000,7.90,45000.0,22.89,0.0,3.0,0.0,0.0,0.0,7181,...,0,0,0,0,0,1,0,0,0,0
1639565,25300,17.99,58584.0,14.07,0.0,0.0,0.0,0.0,0.0,10110,...,0,0,0,0,0,1,0,0,0,0
44669,10000,10.33,90000.0,8.87,0.0,0.0,28.0,0.0,0.0,10817,...,0,0,0,0,0,1,0,0,0,0
2245055,4800,7.21,125000.0,7.56,0.0,0.0,0.0,0.0,0.0,38760,...,0,0,0,0,0,1,0,0,0,0
1884322,4000,19.24,35800.0,23.73,0.0,3.0,64.0,0.0,0.0,6039,...,0,0,0,0,0,1,0,0,0,0


In [26]:
y = df_sample['loan_status']
X = df_sample.drop(['loan_status'], axis=1)

In [27]:
from sklearn.model_selection import train_test_split

y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [28]:
from sklearn import preprocessing as ppg

scaler = ppg.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_train_scaled

array([[ 1.74138432, -1.23373268,  1.60777147, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [-0.01979553,  0.89430734, -0.22887155, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [-1.08101929, -0.03489532, -1.24219184, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       ...,
       [-1.08101929, -0.44971794, -1.02052802, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [-1.36325965, -0.13860097, -0.83053047, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [ 1.28979974, -1.28351139,  0.02445852, ..., -0.0829886 ,
        -0.04491542, -0.08021515]])

In [29]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[ 0.18341753,  0.191183  , -1.24219184, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [ 0.38663059, -0.08674815, -0.52970101, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [-0.29074628, -0.05978468, -0.92552925, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       ...,
       [ 1.24464129,  1.41905795,  0.75658243, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [-0.44880088, -0.88320757,  1.14227746, ..., -0.0829886 ,
        -0.04491542, -0.08021515],
       [-1.13746736,  0.23888761, -0.79886421, ..., -0.0829886 ,
        -0.04491542, -0.08021515]])

# Logistic Regression

In [30]:
# Using K fold cross validation and Grid search to get the most accurate prediction

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]

param_grid = {'C': Cs}
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)
logreg_cv.fit(X_train_scaled, y_train)
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))
# Print the accuracy from the testing data.
print(accuracy_score(logreg_cv.predict(X_test_scaled), y_test))

#clf = LogisticRegression(max_iter=200)
# Fit the model on the trainng data.
#clf.fit(X_train_scaled, y_train)
#y_pred_log = clf.predict(X_test_scaled)
# Print the accuracy from the testing data.
#print(accuracy_score(y_pred_log, y_test))

Tuned Logistic Regression Parameters: {'C': 100}
Best score is 0.97462
0.97306


In [31]:
from sklearn.metrics import confusion_matrix

y_pred_log = logreg_cv.predict(X_test_scaled)
pl_log = confusion_matrix(y_test,y_pred_log)
pl_log

array([[ 5781,    11,     0,     3,   111,     1,     0,     2],
       [    2, 19905,     0,     0,   382,     0,     3,     9],
       [    7,     0,     4,     4,     1,     0,     0,     0],
       [    0,     0,     2,    18,    14,     0,     0,     0],
       [   16,    22,     0,     8, 22907,     1,     0,     0],
       [    0,   207,     0,     0,     3,     0,     0,     3],
       [    0,    76,     0,     0,     1,     0,     0,     2],
       [    0,   449,     0,     0,     6,     1,     0,    38]],
      dtype=int64)

In [32]:
from sklearn.metrics import classification_report

cl_rep_log = classification_report(y_test,y_pred_log)
print(cl_rep_log)

                                                     precision    recall  f1-score   support

                                        Charged Off       1.00      0.98      0.99      5909
                                            Current       0.96      0.98      0.97     20301
Does not meet the credit policy. Status:Charged Off       0.67      0.25      0.36        16
 Does not meet the credit policy. Status:Fully Paid       0.55      0.53      0.54        34
                                         Fully Paid       0.98      1.00      0.99     22954
                                    In Grace Period       0.00      0.00      0.00       213
                                  Late (16-30 days)       0.00      0.00      0.00        79
                                 Late (31-120 days)       0.70      0.08      0.14       494

                                           accuracy                           0.97     50000
                                          macro avg       0.61      

In [41]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

ac = accuracy_score(y_test, y_pred_log)
print('Test Data Set Logistic Regression: Accuracy=%.3f' % (ac))

f1 = f1_score(y_test, y_pred_log, average='weighted')
print('Test Data Set Logistic Regression: f1-score=%.3f' % (f1))

roc_auc = roc_auc_score(y_test, y_pred_log)
print('Test Data Set Logistic Regression: roc_auc_score=%.3f' % (roc_auc))

Test Data Set Logistic Regression: Accuracy=0.973
Test Data Set Logistic Regression: f1-score=0.966


ValueError: could not convert string to float: 'Current'

# Question - Create points plot similar to Case study?  

# Question - Other metrics - ROC_AUC, accuracy, log-loss

# Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=30, random_state = 1,n_jobs=-1)
model_res = clf.fit(X_train_scaled, y_train)
y_pred_ranf = model_res.predict(X_test_scaled)
print(accuracy_score(y_test, y_pred_ranf))

0.97146


In [34]:
y_pred_ranf

array(['Current', 'Current', 'Charged Off', ..., 'Current', 'Fully Paid',
       'Fully Paid'], dtype=object)

In [35]:
pl_ranf = confusion_matrix(y_test,y_pred_ranf)
pl_ranf

array([[ 5447,     0,     0,     0,   462,     0,     0,     0],
       [    1, 20163,     0,     0,   134,     0,     0,     3],
       [   14,     0,     2,     0,     0,     0,     0,     0],
       [    0,     0,     0,    13,    21,     0,     0,     0],
       [   38,     0,     0,     1, 22915,     0,     0,     0],
       [    1,   210,     0,     0,     1,     0,     0,     1],
       [    0,    76,     0,     0,     1,     0,     0,     2],
       [    2,   458,     0,     0,     1,     0,     0,    33]],
      dtype=int64)

In [36]:
cl_rep_ranf = classification_report(y_test,y_pred_ranf)
print(cl_rep_ranf)

                                                     precision    recall  f1-score   support

                                        Charged Off       0.99      0.92      0.95      5909
                                            Current       0.96      0.99      0.98     20301
Does not meet the credit policy. Status:Charged Off       1.00      0.12      0.22        16
 Does not meet the credit policy. Status:Fully Paid       0.93      0.38      0.54        34
                                         Fully Paid       0.97      1.00      0.99     22954
                                    In Grace Period       0.00      0.00      0.00       213
                                  Late (16-30 days)       0.00      0.00      0.00        79
                                 Late (31-120 days)       0.85      0.07      0.12       494

                                           accuracy                           0.97     50000
                                          macro avg       0.71      

# Gradient Boosting

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_scaled, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test_scaled, y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.795
Accuracy score (validation): 0.791

Learning rate:  0.1
Accuracy score (training): 0.831
Accuracy score (validation): 0.827

Learning rate:  0.25
Accuracy score (training): 0.905
Accuracy score (validation): 0.905

Learning rate:  0.5
Accuracy score (training): 0.786
Accuracy score (validation): 0.782

Learning rate:  0.75
Accuracy score (training): 0.749
Accuracy score (validation): 0.746

Learning rate:  1
Accuracy score (training): 0.009
Accuracy score (validation): 0.009



In [38]:
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.25, max_features=2, max_depth = 2, random_state = 0)
gb.fit(X_train_scaled, y_train)
y_pred_gdbst = gb.predict(X_test_scaled)
print(accuracy_score(y_test, y_pred_gdbst))

0.9047


In [39]:
pl_gbst = confusion_matrix(y_test,y_pred_gdbst)
pl_gbst

array([[ 3897,   897,     0,     0,  1112,     0,     0,     3],
       [    4, 19859,     0,     1,   431,     2,     1,     3],
       [    3,     0,     0,     5,     8,     0,     0,     0],
       [    0,     0,     0,     6,    28,     0,     0,     0],
       [   10,  1478,     0,     3, 21462,     0,     0,     1],
       [    1,   205,     0,     0,     4,     0,     1,     2],
       [    0,    73,     0,     0,     4,     0,     2,     0],
       [   20,   445,     0,     0,    13,     2,     5,     9]],
      dtype=int64)

In [40]:
cl_rep_gbst = classification_report(y_test,y_pred_gdbst)
print(cl_rep_gbst)

                                                     precision    recall  f1-score   support

                                        Charged Off       0.99      0.66      0.79      5909
                                            Current       0.87      0.98      0.92     20301
Does not meet the credit policy. Status:Charged Off       0.00      0.00      0.00        16
 Does not meet the credit policy. Status:Fully Paid       0.40      0.18      0.24        34
                                         Fully Paid       0.93      0.94      0.93     22954
                                    In Grace Period       0.00      0.00      0.00       213
                                  Late (16-30 days)       0.22      0.03      0.05        79
                                 Late (31-120 days)       0.50      0.02      0.04       494

                                           accuracy                           0.90     50000
                                          macro avg       0.49      

# Compare Models

Based on the accuracy score and classification report, Logistic regression using 5 fold cross validation and Grid Search performed the best.

# Question: Use 5 fold cross validation and grid search for the last two models?

# Question: Create a table showing f1 score, ROC_AUC, log-loss for all models?

# Create and compare models using pycaret